In [137]:
import numpy as np
from numpy.linalg import det
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time


## 2D SciPy Delaunay

In [786]:

start_time = time.time()
np.random.seed(114131)
# points = np.array([[0,0],[0,1.1],[1,0],[1,1]])
points = np.random.rand(100, 2)

tri1 = Delaunay(points)

print(points[tri1.simplices][1])
plt.triplot(points[:,0], points[:,1], tri1.simplices,)

plt.plot(points[:,0], points[:,1], 'o')

plt.show()

print("--- %s seconds ---" % (time.time() - start_time))

[[0.16103031 0.7345475 ]
 [0.08859133 0.80181665]
 [0.00999027 0.71592058]]
--- 0.21357274055480957 seconds ---


## 3D SciPy Delaunay

In [787]:
def get_first_value(tuples_list):

    return [t[0] for t in tuples_list]
   

def get_second_value(tuples_list):
        return [t[1] for t in tuples_list]
    

def get_third_value(tuples_list):

        return [t[2] for t in tuples_list]

%matplotlib

start_time = time.time()
# Define the function that describes the surface
def f(x, y):
    # return np.cos(.8*np.sqrt(x**2 + y**2)+2)
    # return np.cos(.3*x + 2)
    return np.sin(x*np.pi/10.) * np.cos(y*np.pi*3/20.)

# Define the x, y coordinates of the points on the surface
x = np.linspace(1, 15, 9)
y = np.linspace(1, 15, 9)

# Create a meshgrid from the x, y coordinates
X, Y = np.meshgrid(x, y)

# Calculate the corresponding z coordinates using the surface function
Z = f(X, Y)

# Combine the x, y, and z coordinates into a 3D array of points
points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
################################################################################

tri = Delaunay(points) 


################################################################################
# 3D

# Plot the results (optional)
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(points[:,0],points[:,1],points[:,2])
print("--- %s seconds ---" % (time.time() - start_time))

Using matplotlib backend: MacOSX
--- 0.06795597076416016 seconds ---


# 2D and 3D Delaunay Function

In [673]:
def delaunay_2D(points):
    edges = []
    i = 0
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            for k in range(j+1, len(points)):
                # Check if the three points form a circle that contains no other points

                x1, y1 = points[i]
                x2, y2 = points[j]
                x3, y3 = points[k]
                a = x2 - x1
                b = y2 - y1
                c = x3 - x1
                d = y3 - y1
                e = a*(x1 + x2) + b*(y1 + y2)
                f = c*(x1 + x3) + d*(y1 + y3)
                g = 2*(a*(y3 - y2) - b*(x3 - x2))
                if g == 0:
                    continue
                center_x = (d*e - b*f) / g
                center_y = (a*f - c*e) / g
                radius = ((x1 - center_x)**2 + (y1 - center_y)**2)**0.5
                contains_point = False
                for p in points:
                    if p == points[i] or p == points[j] or p == points[k]:
                        continue
                    if ((p[0] - center_x)**2 + (p[1] - center_y)**2)**0.5 < radius:
                        contains_point = True
                        break
                if not contains_point:
                    edges.append((i, j))
                    edges.append((j, k))
                    edges.append((k, i))
    
    # Remove duplicate edges
    edges = list(set(edges))
    
    return edges
    

In [782]:
def delaunay_3D(points):

        
    # Create a list of tetrahedrons
    tetrahedrons = []
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            for k in range(j+1, len(points)):
                for l in range(k+1, len(points)):
                    
                    # Check if the four points form a circumsphere that contains no other points
                    circumsphere = _circumsphere(points[i], points[j], points[k], points[l])
                    if all(_point_in_sphere(point, circumsphere) for point in points if point not in (points[i], points[j], points[k], points[l])):
                        tetrahedrons.append((points[i], points[j], points[k], points[l]))
    
    return tetrahedrons

def _circumsphere(p1, p2, p3, p4):
  
    # Calculate the coefficients of the matrix
    a = p2[0] - p1[0]
    b = p2[1] - p1[1]
    c = p2[2] - p1[2]
    d = p3[0] - p1[0]
    e = p3[1] - p1[1]
    f = p3[2] - p1[2]
    g = p4[0] - p1[0]
    h = p4[1] - p1[1]
    i = p4[2] - p1[2]
    D = 2 * (a * (e * i - h * f) - b * (d * i - g * f) + c * (d * h - g * e))
    
    # Calculate the center of the circumsphere
    x = ((e * i - h * f) * (a**2 + b**2 + c**2) - (d * i - g * f) * (b**2 + c**2 + (p3[1] - p1[1])**2) + (d * h - g * e) * (c**2 + (p3[2] - p1[2])**2 + (p3[0] - p1[0])**2)) / D
    y = ((d * i - g * f) * (a**2 + b**2 + c**2) - (e * i - h * f) * (a**2 + c**2 + (p2[0] - p1[0])**2) + (d * h - g * e) * ((p3[1] - p1[1])**2 + (p3[0] - p1[0])**2 + c**2)) / D
    z = ((d * h - g * e) * (a**2 + b**2 + c**2) - (e * i - h * f) * (b**2 + c**2 + (p2[1] - p1[1])**2) + (d * i - g * f) * ((p3[2] - p1[2])**2 + (p3[0] - p1[0])**2 + b**2)) / D
    
    # Calculate the radius of the circumsphere
    r = math.sqrt((x - p1[0])**2 + (y - p1[1])**2 + (z - p1[2])**2)
    
    # Return the center and radius of the circumsphere
    return (x, y, z, r)
  


def _point_in_sphere(point, sphere):
    return math.sqrt((point[0]-sphere[0])**2 + (point[1]-sphere[1])**2 + (point[2]-sphere[2])**2) < sphere[1]


# Testing Area

### 2D

In [794]:
def get_first_value(tuples_list):

    return [t[0] for t in tuples_list]

def get_second_value(tuples_list):

    return [t[1] for t in tuples_list]
   

def get_third_value(tuples_list):

    return [t[2] for t in tuples_list]
   
%matplotlib

start_time = time.time()


################################################################################

np.random.seed(114131)
points = np.random.rand(100,2)


################################################################################
D = 2

points.tolist()
points1 = []
for i in range(len(points)):
    if D == 2:
        points1.append((points[i,0],points[i,1]))
        tri = delaunay_2D(points1) 
    else:
        points1.append((points[i,0],points[i,1],points[i,2]))
        tri = delaunay_3D(points1) 

print("--- %s seconds ---" % (time.time() - start_time))


###############################################################################
# 2D
index = get_first_value(tri)

plt.triplot(points[index][:,0], points[index][:,1])

plt.show()



print("--- %s seconds ---" % (time.time() - start_time))

Using matplotlib backend: MacOSX
--- 20.30254626274109 seconds ---
--- 20.37178897857666 seconds ---


### 3D

In [795]:
def get_first_value(tuples_list):

    return [t[0] for t in tuples_list]

def get_second_value(tuples_list):

    return [t[1] for t in tuples_list]
   

def get_third_value(tuples_list):

    return [t[2] for t in tuples_list]
   
%matplotlib

start_time = time.time()


################################################################################

# Define the function that describes the surface
def f(x, y):
    # return np.cos(.8*np.sqrt(x**2 + y**2)+2)
    # return np.cos(.3*x + 2)
    # return np.sqrt(2-(x-2)**2-y**2) +2
    return np.sin(x*np.pi/10.) * np.cos(y*np.pi*3/20.)

# Define the x, y coordinates of the points on the surface
x = np.linspace(1, 15, 5)
y = np.linspace(1, 15, 5)

# Create a meshgrid from the x, y coordinates
X, Y = np.meshgrid(x, y)

# Calculate the corresponding z coordinates using the surface function
Z = f(X, Y)

# Combine the x, y, and z coordinates into a 3D array of points
points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
################################################################################

################################################################################
D = 3

points.tolist()
points1 = []
for i in range(len(points)):
    if D == 2:
        points1.append((points[i,0],points[i,1]))
        tri = delaunay_2D(points1) 
    else:
        points1.append((points[i,0],points[i,1],points[i,2]))
        tri = delaunay_3D(points1) 

print("--- %s seconds ---" % (time.time() - start_time))
################################################################################
# 3D
index = [sp for tpl in tri for sp in tpl]

x_index = get_first_value(index)
y_index = get_second_value(index)
z_index = get_third_value(index)


# Plot the results (optional)
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(x_index, y_index, z_index)



print("--- %s seconds ---" % (time.time() - start_time))

Using matplotlib backend: MacOSX


/var/folders/1p/cp6mx14s7fbcxp3s3vtxppl00000gn/T/ipykernel_11698/3520562298.py:33: RuntimeWarning: divide by zero encountered in double_scalars
  x = ((e * i - h * f) * (a**2 + b**2 + c**2) - (d * i - g * f) * (b**2 + c**2 + (p3[1] - p1[1])**2) + (d * h - g * e) * (c**2 + (p3[2] - p1[2])**2 + (p3[0] - p1[0])**2)) / D
/var/folders/1p/cp6mx14s7fbcxp3s3vtxppl00000gn/T/ipykernel_11698/3520562298.py:34: RuntimeWarning: divide by zero encountered in double_scalars
  y = ((d * i - g * f) * (a**2 + b**2 + c**2) - (e * i - h * f) * (a**2 + c**2 + (p2[0] - p1[0])**2) + (d * h - g * e) * ((p3[1] - p1[1])**2 + (p3[0] - p1[0])**2 + c**2)) / D
/var/folders/1p/cp6mx14s7fbcxp3s3vtxppl00000gn/T/ipykernel_11698/3520562298.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  z = ((d * h - g * e) * (a**2 + b**2 + c**2) - (e * i - h * f) * (b**2 + c**2 + (p2[1] - p1[1])**2) + (d * i - g * f) * ((p3[2] - p1[2])**2 + (p3[0] - p1[0])**2 + b**2)) / D
/var/folders/1p/cp6mx14s7fbcxp3s3vtxppl0000

--- 0.8620121479034424 seconds ---
--- 0.9547891616821289 seconds ---
